# Analysis: Illusory Pitch

In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
import scipy.stats as stats
import matplotlib.pyplot as plt

# Set participants to exclude
EXCLUDED = [15, 22]

# Set file paths
TRIAL_DATAFILE = '../data/response_data.csv'
SCORES_DATAFILE = '../data/scores.csv'
SUBJ_SCORES_DATAFILE = '../data/subj_scores.csv'
FIGURE_PATH = './figures/'

### Define figure style

In [ ]:
THEME = 'light'

# Set line widths and marker sizes
lw = 1.5
capsize = 3
msize = 7

# Set colors for light and dark versions of figures
if THEME == 'dark':
    background_color = '#151619'
    text_color = 'w'
    color1 = '#90E6E2'
    color2 = '#FAB2B9'
elif THEME == 'light':
    background_color = 'w'
    text_color = 'k'
    color1 = '#512D6D'
    color2 = '#F8485E'
else:
    raise ValueError('Theme not recognized - must be one of "light" or "dark".')

plt.rcParams.update({
    # BACKGROUND
    'figure.facecolor': background_color,
    'axes.facecolor': background_color,
    'savefig.facecolor': background_color,

    # TEXT
    'text.color': text_color,
    'axes.labelcolor': text_color,
    'xtick.color': text_color,
    'ytick.color': text_color,
    'figure.titlesize': 20,  # fontsize of the figure title
    'axes.titlesize': 20,
    'axes.labelsize': 16,  # fontsize of the x and y labels
    'xtick.labelsize': 12,  # fontsize of the tick labels
    'ytick.labelsize': 12,  # fontsize of the tick labels
    'legend.fontsize': 12,  # legend fontsize
    'legend.title_fontsize': 14,  # legend title size
    'font.size': 12,  # text size

    # FIGURE STYLE
    'axes.spines.top': False,
    'axes.spines.right': False,
    'axes.spines.left': True,
    'axes.spines.bottom': True,
    'axes.edgecolor': text_color,
    'axes.grid': False,
    'ytick.left': True,
    'xtick.bottom': True
})

### Load and preprocess data

In [ ]:
# Load scores
scores = pd.read_csv(SCORES_DATAFILE)
scores = scores[~np.isin(scores.subject, EXCLUDED)]

# Loftus & Masson (1994) method of plotting within-subject effects
subj_means = scores.groupby(['subject']).mean().reset_index()
grand_mean = subj_means.mean()
within_scores = scores.copy()
for i, subj in enumerate(subj_means.subject):
    within_scores.loc[within_scores.subject == subj, 'dprime'] -= subj_means.dprime[i] - grand_mean.dprime
    within_scores.loc[within_scores.subject == subj, 'C'] -= subj_means.C[i] - grand_mean.C

# Stagger the x-axis values so error bars will be easier to read
within_scores.loc[within_scores.octave == 3, 'offset'] -= .5
within_scores.loc[within_scores.octave == 5, 'offset'] += .5

### Plot bias and sensitivity

In [ ]:
###
# Bias
###

plt.subplot(121)

# Plot average C per condition
sns.lineplot(x='offset', y='C', hue='octave', data=within_scores,
             hue_order=[3, 5], palette=[color1, color2],
             ls='-', lw=lw, marker='o', ms=msize,
             err_style='bars', err_kws=dict(capsize=capsize, lw=lw, capthick=lw))
plt.axhline(0, ls='--', c='k', alpha=.5)


# Stylize subplot
plt.legend(['A3 (220 Hz)', 'A5 (880 Hz)'], title='Standard Pitch', loc='upper left', framealpha=1)
plt.xlim(-20, 20)
plt.xticks([-15, 0, 15], ['15% Early', 'On Time', '15% Late'])
plt.ylim(-.6, 1)
plt.xlabel('Probe Timing Offset')
plt.ylabel('Bias ($C$) Towards "Low"')


###
# Sensitivity
###

plt.subplot(122)

# Plot average d' per condition
sns.lineplot(x='offset', y='dprime', hue='octave', data=within_scores,
             hue_order=[3, 5], palette=[color1, color2],
             ls='-', lw=lw, marker='o', ms=msize,
             err_style='bars', err_kws=dict(capsize=capsize, lw=lw, capthick=lw))
plt.axhline(0, ls='--', c='k', alpha=.5)


# Stylize subplot
plt.legend(['A3 (220 Hz)', 'A5 (880 Hz)'], title='Standard Pitch', loc='lower right', framealpha=1)
plt.xlim(-20, 20)
plt.xticks([-15, 0, 15], ['15% Early', 'On Time', '15% Late'])
plt.ylim(-.5, 3)
plt.xlabel('Probe Timing Offset')
plt.ylabel('Sensitivity ($d\prime$)')

# Stylize figure and save
plt.gcf().set_size_inches(9, 4)
plt.tight_layout()
plt.gcf().savefig(FIGURE_PATH + 'scores.svg')

### Plot reaction time

In [ ]:
# Load trial data
data = pd.read_csv(TRIAL_DATAFILE)
data = data[~np.isin(data.subject, EXCLUDED)]

# Exclude trials with reaction times slower than 10 seconds
bad_rt = data.rt > 10000
print('Excluding %s percent of trials for outlier RT' % (bad_rt.mean() * 100))
data = data[~bad_rt]

# Loftus & Masson (1994) method of plotting within-subject effects
# For reaction time plots, we will analyze at the trial level rather than subject level due to some participants 
# having very few incorrect responses, giving us poor estimates of their mean reaction times for incorrect trials
subj_means = data.groupby('subject').mean().reset_index()
grand_mean = subj_means.mean()
within_data = data.copy()
for i, subj in enumerate(subj_means.subject):
    within_data.loc[within_data.subject == subj, 'rt'] -= subj_means.rt[i] - grand_mean.rt

# Stagger the x-axis values so error bars will be easier to read
within_data.loc[within_data.pitch_shift == '+', 'offset'] -= .5
within_data.loc[within_data.pitch_shift == '-', 'offset'] += .5

In [ ]:
plt.subplot(121)

sns.lineplot(x='offset', y='rt', hue='pitch_shift', data=within_data[within_data['correct']],
             ls='-', lw=lw, marker='o', ms=msize, hue_order=['+', '-'], palette=[color1, color2],
             err_style='bars', err_kws=dict(capsize=capsize, lw=lw, capthick=lw))

plt.title('Correct Response')
plt.xlabel('Probe Timing Offset')
plt.ylabel('Reaction Time (ms)')
plt.legend(['Up', 'Down'], title='Pitch Shift', loc='upper left')
plt.xlim(-20, 20)
plt.xticks([-15, 0, 15], ['15% Early', 'On Time', '15% Late'])
plt.ylim(700, 1300)

plt.subplot(122)
sns.lineplot(x='offset', y='rt', hue='pitch_shift', data=within_data[~within_data['correct']],
             ls='-', lw=lw, marker='o', ms=msize, hue_order=['+', '-'], palette=[color1, color2],
             err_style='bars', err_kws=dict(capsize=capsize, lw=lw, capthick=lw))

# Stylize subplot
plt.title('Incorrect Response')
plt.xlabel('Probe Timing Offset')
plt.ylabel('Reaction Time (ms)')
plt.legend(['Up', 'Down'], title='Pitch Shift', loc='upper right')
plt.xlim(-20, 20)
plt.xticks([-15, 0, 15], ['15% Early', 'On Time', '15% Late'])
plt.ylim(1000, 2000)

# Stylize figure and save
plt.gcf().set_size_inches(10, 4.5)
plt.tight_layout()
plt.gcf().savefig(FIGURE_PATH + 'rt.svg')

### Plot sensitivity and bias correlation

In [ ]:
# Load subject data
sdata = pd.read_csv(SUBJ_SCORES_DATAFILE)
sdata = sdata[~np.isin(sdata.subject, EXCLUDED)]

In [ ]:
sns.regplot(x='dprime', y='C_slope', data=sdata, color=text_color)
plt.axhline(y=0, linestyle='--', color=text_color, alpha=.5)
r, p = stats.pearsonr(sdata.dprime, sdata.C_slope)
plt.text(3, .1, '$r=%.03f$, $p=%.03f$' % (r, p))
plt.xlabel('Sensitivity ($d\prime$)')
plt.ylabel('Timing-Related Bias')
plt.ylim(-.03, .11)
plt.gcf().set_size_inches(6, 4.5)
plt.tight_layout()
plt.savefig(FIGURE_PATH + 'correlation.svg')